In [1]:
cd ~/SageMaker/docker_test_folder

/home/ec2-user/SageMaker/docker_test_folder


In [2]:
! pwd

/home/ec2-user/SageMaker/docker_test_folder


In [4]:
%%sh

# Specify an algorithm name
algorithm_name=pytorch-extended-container-test

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Log into Docker
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/5 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.1-cpu-py36-ubuntu16.04
 ---> 6177bf009b27
Step 2/5 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> f17c4b5f50ed
Step 3/5 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> 3fa158fc2179
Step 4/5 : COPY cifar10.py /opt/ml/code/cifar10.py
 ---> Using cache
 ---> 535d94911874
Step 5/5 : ENV SAGEMAKER_PROGRAM cifar10.py
 ---> Using cache
 ---> 9872e012c96f
Successfully built 9872e012c96f
Successfully tagged pytorch-extended-container-test:latest
The push refers to repository [551329315830.dkr.ecr.us-east-1.amazonaws.com/pytorch-extended-container-test]
6792506cf269: Preparing
ea7f95045e04: Preparing
6dffe9708ca1: Preparing
d6988f6060c6: Preparing
e53aa3c12226: Preparing
f74c62daa95e: Preparing
05cd8fe1cb59: Preparing
b35d061a5b0d: Preparing
b6073ff1bf08: Preparing
58defc0e8a35: Preparing
d3b16a8aee3d: Preparing
42fae27443e4: Preparing
5276d2b930fc: Preparing
e6feec

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

